In [62]:
from utils import *
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


In [6]:
rho = np.array([[1.00,  0.10,  0.45,  0.64 * np.sqrt(32.00)],
                [0.10,  1.00, -0.35, -0.24 * np.sqrt(32.00)],
                [0.45, -0.35,  1.00,  0.14 * np.sqrt(32.00)],
                [0.64 * np.sqrt(32.00), -0.24 * np.sqrt(32.00),  0.14 * np.sqrt(32.00), 32.00]])

r = 0.9
betas = np.array([1, 1, 1])
gammas = np.array([[3.3, 0.0, 0.0], # в оригинале 0.9, но мб надо 3.3
                   [1.0, 1.0, 0.0],
                   [1.0, 0.0, 1.0]])

In [21]:
np.random.seed(999)

data = get_benchmark_data(1000, rho, r, betas, gammas)
data.head()

,w0,w1,w2,x0,x1,x2,z_star1,z_star2,z_star3,z,y
0,1.0,3.945854,3.023966,1.0,2.106524,5.384220,10.362618,4.944527,1.662203,1.0,7.829563
1,1.0,3.733443,0.301256,1.0,4.842226,0.143154,9.324524,3.926476,1.012167,1.0,7.274203
2,1.0,2.468385,4.463601,1.0,3.314473,6.098579,8.721487,3.223663,3.317389,1.0,2.283242
3,1.0,-1.670901,-1.538022,1.0,0.264831,-1.790422,0.562982,-2.296950,-0.728739,1.0,5.749075
4,1.0,1.707867,-4.200895,1.0,1.439676,-5.091854,1.480205,3.233472,-5.464949,2.0,NaN


In [74]:
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __getitem__(self, index):
        row = self.dataframe.iloc[index].to_numpy()
        features = row[[1, 2, 4, 5]]
        label = row[-1]
        return features.astype(float), label.astype(float)

    def __len__(self):
        return len(self.dataframe)

df_no_nans = data.dropna()

dataset = CustomDataset(dataframe=df_no_nans)
dataloader = DataLoader(dataset, batch_size=32)

In [75]:
for i in dataloader:
    print(i[0][:, :2])
    break

tensor([[ 3.9459,  3.0240],
        [ 3.7334,  0.3013],
        [ 2.4684,  4.4636],
        [-1.6709, -1.5380],
        [ 1.9291, -0.4015],
        [-1.6359,  6.4257],
        [-0.1219,  1.8537],
        [ 2.1464,  4.8397],
        [-0.4758, -2.5274],
        [ 4.3659,  2.9539],
        [ 1.6330,  5.8968],
        [ 7.4397,  9.3662],
        [-0.4139,  6.4879],
        [ 6.4996,  6.2111],
        [ 2.8034, -2.7818],
        [-4.0844,  4.6878],
        [-0.3246, -0.6593],
        [ 4.4508,  3.6893],
        [ 3.6451,  4.9506],
        [ 0.6681,  3.9657],
        [ 1.3052,  0.6435],
        [-0.9310, -3.5504],
        [-0.3850, -0.7644],
        [-0.7269,  1.5168],
        [ 0.9661,  3.5564],
        [-0.5719,  5.5302],
        [-3.0472,  2.6663],
        [ 5.2588, -2.7872],
        [-1.2293,  1.3259],
        [ 3.6585,  0.7180],
        [ 3.9786,  2.6729],
        [-0.0424, -2.8982]], dtype=torch.float64)


In [114]:
class BASSnet(nn.Module):
    def __init__(self, dim_latent=2):
        super(BASSnet, self).__init__()
        self.fc1 = nn.Linear(2, 32)
        self.fc2 = nn.Linear(32, 32)
        self.fc3 = nn.Linear(32, dim_latent) # выходное пространство
        self.fc4 = nn.Linear(dim_latent+2, 1)
  
    def forward(self, inputs):
        g, x_main = inputs
        g = g.to(torch.float32)
        x_main = x_main.to(torch.float32)
        
        g = F.relu(self.fc1(g))
        g = F.relu(self.fc2(g))
        g = self.fc3(g)
        
        # конкатенируем с регрессорами основного уравнения
        x = torch.cat([g, x_main], dim=1)
        x = self.fc4(x)
        return x
    
model = BASSnet()
loss_fn = nn.MSELoss() 
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [117]:
from tqdm.auto import tqdm

n_epochs = 3
# Hold the best model
best_mse = np.inf   # init to infinity
best_weights = None
history = []
 
# training loop
for epoch in range(n_epochs):
    epoch_history = []
    model.train()
    for X_batch, y_batch in tqdm(dataloader):
        X_selection = X_batch[:, :2]
        X_main = X_batch[:, 2:]

        y_pred = model((X_selection, X_main)).to(torch.float64)
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_history.append(loss.detach().numpy())
    if epoch % 10 == 0:
        print(f'Эпоха {epoch}. MSE:', np.mean(epoch_history))
        
#     # evaluate accuracy at end of each epoch
#     model.eval()
#     y_pred = model(X_test)
#     mse = loss_fn(y_pred, y_test)
#     mse = float(mse)
#     history.append(mse)
#     if mse < best_mse:
#         best_mse = mse
#         best_weights = copy.deepcopy(model.state_dict())
 
# # restore model and return best accuracy
# model.load_state_dict(best_weights)

  0%|          | 0/20 [00:00<?, ?it/s]

Эпоха 0. MSE: 56.382658282521604


  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

In [118]:
model.fc4.weight

Parameter containing:
tensor([[-0.4348,  0.0874, -0.2030, -0.1677]], requires_grad=True)

In [58]:
class BASSnet(nn.Module):
    def __init__(self, dim_latent=10, ):
        super(BASSnet, self).__init__()
        self.fc1 = nn.Linear(3, 32)
        self.fc2 = nn.Linear(32, dim_latent) # выходное пространство
        self.fc3 = nn.Linear(dim_latent+2, 1)
  
    def forward(self, inputs):
        g, x_main = inputs
        g = F.relu(self.fc1(g))
        g = F.relu(self.fc2(g))
#         x = torch.flatten(g, 1) 

        # конкатенируем с регрессорами основного уравнения
        x = torch.cat([g, x_main], dim=1)
        x = self.fc3(x)
        return x
    
    
bassnet = BASSnet()
bassnet

BASSnet(
  (fc1): Linear(in_features=3, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=10, bias=True)
  (fc3): Linear(in_features=12, out_features=1, bias=True)
)